In [38]:
import os 
import pandas as pd
import re

In [39]:
DataFrames = []
for fileName in os.listdir('DataSets'):
    if fileName.endswith('.csv'):
        df = pd.read_csv('DataSets/' + fileName, encoding = 'gbk')
        DataFrames.append(df)

df = pd.concat(DataFrames)
del DataFrames

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
with open('/Users/apricity/Desktop/record/论文_基于高校招生数据分析_20190404/985高校清单.txt') as fr:
    text = fr.read()
    college_list = re.findall('\w+大学', text)
    
def delete_college(college):
    if college in college_list or "医学院" in college:
        return 1
    else:
        return 0
    
df['是否本部及本部附属医学院'] = df['学校'].apply(delete_college)       
df = df.query("是否本部及本部附属医学院 == 1")

df = df.drop_duplicates(subset = ['专业', '学校','考生地区', '文理科', '年份', '批次'])
df = df.drop('多余列', axis = 1)

### 填充缺失字段

In [41]:
def fill_average_score(average_score, high_score):
    if average_score == '--' and high_score != '--':
        average_score = high_score
    return average_score
    
def fill_high_score(average_score, high_score):
    if average_score != '--' and high_score == '--':
        high_score = average_score 
    return high_score


df['平均分'] = df.apply(lambda row : fill_average_score( row['平均分'],
                                                    row['最高分']) , axis = 1)
df['最高分'] = df.apply(lambda row : fill_high_score( row['平均分'],
                                                    row['最高分']) , axis = 1)

df = df.query("最高分 != '--' ")

df["平均分"] = df["平均分"].astype(int)
df["最高分"] = df["最高分"].astype(int)


### 历年各高校专业数量

In [42]:
pd.pivot_table( df, index = ['年份'], columns = ['学校'], values = ['专业'], aggfunc = len, fill_value = 0)

专业                                                                 \
学校   上海交通大学 上海交通大学医学院 东北大学 东南大学  中南大学 中国人民大学 中国农业大学 中国海洋大学 中国科学技术大学 中央民族大学   
年份                                                                           
2007    533         0    0  915     0    816    710    839      376    761   
2008    545         0    0  908     0    804    760    417      378    741   
2009    475        71    0  757   910    730    702    861      386    819   
2010    299        81  714  597   870    615    673    952      358    797   
2011    301        71  702  609   952    684    710    996      374    863   
2012    217        30  664  522   800    510    641    889      258    813   
2013    202         0  355  494   990    410    467    604      202    528   
2014    464         0  994  739  1347    809    746    796      358    412   
2015    129         0  696  528   993    270    307    612      137    225   
2016     80         0  122  358   684    227    246    472       80    176   
2017     43         0   28  175   438    241    479    474       49    181   

      ...                                                                  
学校    ...  山东大学  武汉大学 浙江大学 清华大学  湖南大学 电子科技大学 西北农林科技大学 西北工业大学 西安交通大学  重庆大学  
年份    ...                                                                  
2007  ...     0  1341  578  702  1152      0        0    670      0  1382  
2008  ...     0  1385  281  728  1110    594        0    642      0  1423  
2009  ...     0  1338  161  720  1105    708     1029    674      0  1432  
2010  ...     0  1385  108  612  1116    677     1027    678      0  1387  
2011  ...     0  1438  100  593  1194    530     1044    714      3  1459  
2012  ...     0  1060   91  439  1094    438      795    443    346  1099  
2013  ...     0   800   17  558   873    382      636    530    485   903  
2014  ...   450  1292  122  991  1024    563      795    705    706  1122  
2015  ...   508   696   23  503   760    383      495    450    245   851  
2016  ...   486   573   15  283   600    317      412    405    217   727  
2017  ...   535   377    2   34   693    320      517    431    128   789  

[11 rows x 39 columns]

### 统计 某专业 xx年分文理科在xx省份下xx专业 的排名，及排名得分

In [47]:
def ind2score(ind):
    if m == 1:
        return 1
    else:
        score = (m - ind  ) / (m-1)
        return score
    
DataSet = []
for college in set(df['学校']):
    for province in set(df['考生地区']):
        for year in set(df['年份']):
            for subset in set(df['文理科']):
                mask = (df['学校']==college) & (df['考生地区']==province) & (df['年份']==year) & (df['文理科']==subset) 
                data = df[mask].sort_values(by = '平均分')
                m = len(data)
                data['专业在大学内排名'] = range(1,m+1)
                data['专业在大学内排名得分'] = data['专业在大学内排名'.format(year)].apply(ind2score)
                DataSet.append(data)
                
df = pd.concat(DataSet)    

### 统计 某专业 xx年分文理科在xx省份下xx专业 的排名，及排名得分

In [48]:
DataSet = []

for province in set(df['考生地区']):
    for year in set(df['年份']):
        for subset in set(df['文理科']):
            mask =   (df['考生地区']==province) & (df['年份']==year) & (df['文理科']==subset) 
            data = df[mask].sort_values(by = '平均分')
            m = len(data)
            data['专业在省内排名'] = range(1,m+1)
            data['专业在省内排名得分'] = data['专业在省内排名'].apply(ind2score)
            DataSet.append(data)
            
df = pd.concat(DataSet) 

In [53]:
# df.to_csv('汇总数据.csv', index = None , encoding = 'gbk')